# MLOPT Knapsack Example

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import logging

import mlopt
from mlopt.sampling import uniform_sphere_sample
from mlopt.learners.pytorch.pytorch import PyTorchNeuralNet
from mlopt.utils import n_features, pandas2array

## Generate problem data

In [2]:
np.random.seed(1)  # Reset random seed for reproducibility

# Variable
n = 10
x = cp.Variable(n, integer=True)

# Cost
c = np.random.rand(n)

# Weights
a = cp.Parameter(n, nonneg=True, name='a')
x_u = cp.Parameter(n, nonneg=True, name='x_u')
b = 0.5 * n

## Create optimizer object

In [ ]:
# Problem
cost = - c * x
constraints = [a * x <= b,
               0 <= x, x <= x_u]


# Define optimizer
# If you just want to remove too many messages
# change INFO to WARNING
m = mlopt.Optimizer(cp.Minimize(cost), constraints,
                    log_level=logging.INFO, verbose=True)

> /Users/sidereus/Dropbox/research/code/projects/learning_opt/mlopt/mlopt/problem.py(67)__init__()
     66         # Set options
---> 67         self.tight_constraints = tight_constraints
     68         self.solver_options = solver_options



ipdb>  solver_options


{'verbose': True, 'Method': 1}


## Define training and testing parameters

In [4]:
# Average request
theta_bar = 2 * np.ones(2 * n)
radius = 1.0


def sample(theta_bar, radius, n=100):

    # Sample points from multivariate ball
    ndim = int(len(theta_bar)/2)
    X_a = uniform_sphere_sample(theta_bar[:ndim], radius, n=n)
    X_u = uniform_sphere_sample(theta_bar[ndim:], radius, n=n)

    df = pd.DataFrame({
        'a': list(X_a),
        'x_u': list(X_u)
        })

    return df


# Training and testing data
n_train = 1000
n_test = 100
theta_train = sample(theta_bar, radius, n=n_train)
theta_test = sample(theta_bar, radius, n=n_test)

## Train predictor (Pytorch)

In [5]:
# Dictionary of different parameters.
# The cross validation will try all of the possible
# combinations
params = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32],
    'n_epochs': [10]
}
m.train(theta_train, learner=mlopt.PYTORCH, params=params)

100%|██████████| 1000/1000 [00:27<00:00, 36.43it/s]
INFO:mlopt:Encoding strategies
INFO:mlopt:Getting unique set of strategies
INFO:mlopt:Found 43 unique strategies
INFO:mlopt:Caching KKT solver factors for each strategy (it works only for QP-representable problems with parameters only in constraints RHS)
100%|██████████| 43/43 [00:01<00:00, 42.89it/s]
INFO:mlopt:Using CPU with Pytorch
Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 2110.54it/s]
INFO:mlopt:Split dataset in 900 training and 100 validation
INFO:mlopt:Train Neural Network with 3 sets of parameters, 20 inputs, 43 outputs
INFO:mlopt:Learning Neural Network with parameters: {'learning_rate': 0.001, 'batch_size': 32, 'n_epochs': 10, 'n_hidden': 31}
INFO:mlopt:Epoch 1/10
100%|██████████| 29/29 [00:00<00:00, 51.57it/s, loss=3.668]
INFO:mlopt:- Train metrics: accuracy: 0.000 ; loss: 3.729
INFO:mlopt:- Eval metrics: accuracy: 0.400 ; loss: 3.594
INFO:mlopt:Epoch 2/10
100%|██████████| 29/29 [00:00<00:00, 95

## Benchmark on testing dataset

In [6]:
results = m.performance(theta_test)
print("Accuracy: %.2f " % results[0]['accuracy'])

INFO:root:Performance evaluation
INFO:root:Compute tight constraints for test set (n_jobs = 1)
  0%|          | 0/1000 [00:00<?, ?it/s]

Academic license - for non-commercial use only


INFO:gurobipy:Academic license - for non-commercial use only
Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 6836.02it/s]
INFO:root:Predict tight constraints for test set
100%|██████████| 1000/1000 [01:49<00:00,  9.15it/s]


Accuracy: 98.00 


## Save training data

In [7]:
m.save_training_data("training_data.pkl", delete_existing=True)

## Create new solver and train passing loaded data

In [8]:
m = mlopt.Optimizer(cp.Minimize(cost), constraints)
m.load_training_data("training_data.pkl")
m.train(learner=mlopt.PYTORCH, params=params)  # Train after loading samples

results = m.performance(theta_test)
print("Accuracy: %.2f " % results[0]['accuracy'])

INFO:root:Caching KKT solver factors for each strategy (it works only for QP-representable problems with parameters only in constraints RHS)
100%|██████████| 50/50 [00:00<00:00, 90.66it/s]
INFO:root:Using CPU with Pytorch
Converting dataframe to array: 100%|██████████| 10000/10000 [00:01<00:00, 6772.84it/s]
INFO:root:Split dataset in 9000 training and 1000 validation
INFO:root:Train Neural Network with 3 sets of parameters, 20 inputs, 50 outputs
INFO:root:Learning Neural Network with parameters: {'learning_rate': 0.001, 'batch_size': 32, 'n_epochs': 10, 'n_hidden': 35}
INFO:root:Epoch 1/10
100%|██████████| 282/282 [00:01<00:00, 204.66it/s, loss=2.120]
INFO:root:- Train metrics: accuracy: 0.333 ; loss: 2.669
INFO:root:- Eval metrics: accuracy: 0.553 ; loss: 1.697
INFO:root:Epoch 2/10
100%|██████████| 282/282 [00:01<00:00, 165.72it/s, loss=1.399]
INFO:root:- Train metrics: accuracy: 0.594 ; loss: 1.664
INFO:root:- Eval metrics: accuracy: 0.687 ; loss: 1.336
INFO:root:Epoch 3/10
100%|████

Accuracy: 98.00 


## Predict single point

In [13]:
# Predict single point
theta = theta_test.iloc[0]
root = logging.getLogger('mlopt')
root.setLevel(logging.DEBUG)
result_single_point = m.solve(theta)
print(result_single_point)

Converting dataframe to array: 100%|██████████| 1/1 [00:00<00:00, 1078.78it/s]
INFO:root:Predict optimal solution
  0%|          | 0/1 [00:00<?, ?it/s]DEBUG:root:Solving 27 x 27 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 27 x 27 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 27 x 27 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 27 x 27 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 27 x 27 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 27 x 27 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 28 x 28 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 28 x 28 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 28 x 28 linear system A x = b 
DEBUG:root:Using KKT solver cache
DEBUG:root:Solving 28 x 28 linear system A x = b 
DEBUG:root:Using KKT solver cache
100%|████

{'x': array([0., 1., 0., 0., 0., 0., 0., 1., 1., 0.]), 'time': 0.008311986923217773, 'strategy': Strategy
  - Tight constraints:
         id: elements
          7: False
         11: [ True False  True  True  True  True  True False False  True]
         15: [False False False False False False False False False False]
  - Integer variables values:
         id: elements
          0: [0 1 0 0 0 0 0 1 1 0], 'cost': -1.4626526947158758, 'infeasibility': 0.0, 'pred_time': 0.0011489391326904297, 'solve_time': 0.007163047790527344}


## Learn directly from points (talk directly to pytorch)

In [13]:
y = m.y_train
X = m.X_train
learner = PyTorchNeuralNet(n_input=n_features(X),
                           n_classes=len(np.unique(y)),
                           n_best=3,
                           params=params)
# Train learner
learner.train(pandas2array(X), y)

# Predict
X_pred = X.iloc[0]
y_pred = learner.predict(pandas2array(X_pred))  # n_best most likely classes

INFO:root:Using CPU with Pytorch
Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 4045.27it/s]
INFO:root:Split dataset in 900 training and 100 validation
INFO:root:Train Neural Network with 2 sets of parameters, 20 inputs, 43 outputs
INFO:root:Learning Neural Network with parameters: {'learning_rate': 0.001, 'batch_size': 132, 'n_epochs': 10, 'n_hidden': 31}
INFO:root:Epoch 1/10
100%|██████████| 7/7 [00:00<00:00, 120.63it/s, loss=3.698]
INFO:root:- Train metrics: accuracy: 0.205 ; loss: 3.713
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.684
INFO:root:Epoch 2/10
100%|██████████| 7/7 [00:00<00:00, 182.80it/s, loss=3.630]
INFO:root:- Train metrics: accuracy: 0.394 ; loss: 3.661
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.593
INFO:root:Epoch 3/10
100%|██████████| 7/7 [00:00<00:00, 92.05it/s, loss=3.522] 
INFO:root:- Train metrics: accuracy: 0.394 ; loss: 3.570
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.462
INFO:root:Epoch 4/10
100%|██████████|